# Solution of Intro LAB

In this example a simple linear regression is performed with scipy and scikit learn. The implementation of the least square algorithm is also presented.

In [ ]:
#import all required packages
import matplotlib.pyplot as plt
import csv
from scipy import stats
import numpy as np
import sklearn as sl
from sklearn import linear_model

In [ ]:
#read used car data from file
filename = "data/km_year_power_price.csv"
lines = csv.reader(open(filename, newline=''), delimiter=',')

print(type(lines))

data=list(lines)
for line in data[0:10]:
    print(line)

In [ ]:
with open(filename, 'r') as f:
    reader = csv.reader(f, delimiter=',')
    # get header from first row
    headers = next(reader)
    # get all the rows as a list
    data = list(reader)
    # transform data into numpy array
    data = np.array(data).astype(float)
    
print(headers)
print(data.shape)
print(data[:3])

In [ ]:
# linear regression with linregress (estimate price from year)
slope, intercept, r_value, p_value, std_err = stats.linregress(data[:,1], data[:,3])

print('slope (linregress): ', slope,'  intercept (linregress):', intercept);
print('correlation coefficient:', r_value)

In [ ]:
# linear regression with linear_model.LinearRegression() (estimate price from year)
# The fit method requires X to have shape (n_samples, n_features) and Y (n_samples, n_targets)
# Even if we have 1 feature and 1 target recall that in Python a mx1 matrix is different from a 1D array -> need to reshape
reg = sl.linear_model.LinearRegression().fit(data[:,1].reshape(-1,1), data[:,3].reshape(-1,1))

print('slope (sl.linear_model): ', reg.coef_,'  intercept (sl.linear_model):', reg.intercept_);
# The coefficient of determination R^2 is equal to the correlation coefficient squared r^2 only under some very specific conditions
print('correlation coefficient:', np.sqrt(reg.score(data[:,1].reshape(-1,1), data[:,3].reshape(-1,1))))

In [ ]:
# linear regression with least squares (estimate price from year)

n = len(data[:,1]);

vx = np.zeros((n,2),);
vx[:,0] = 1; # we use homogeneous coordinates
vx[:,1] = data[:,1];
vy = data[:,3];

A = np.zeros((2,2))
b = np.zeros((2,1))
for i in range(0,n):
    e = vx[i,:].reshape(2,1)  # 1D array is different from 2x1 matrix
    A = A + np.matmul(e,np.transpose(e))
    b = b + vy[i]*e

w = np.matmul(np.linalg.inv(A),b); # solve least squares

print('slope (least squares): ', w[1],'  intercept (least squares):', w[0]);

In [ ]:
# plot the data, the 3 methods almost perfectly overlap
plt.plot(data[:,1], data[:,3], 'o', label='original data')
plt.plot(data[:,1], intercept + slope*data[:,1], 'r*', label='fitted line linregress')
plt.plot(data[:,1], float(reg.intercept_) + float(reg.coef_)*data[:,1], 'y+', label='fitted line Linear Model')
plt.plot(data[:,1], w[0] + w[1]*data[:,1], 'g', label='fitted line Least Squares')
plt.legend()
plt.xlabel('Year')
plt.ylabel('Price [$]')
plt.show()

print('correlation coefficient wrt the year:', r_value) 

In [ ]:
# linear regression with linregress (estimate price from power)

slope, intercept, r_value, p_value, std_err = stats.linregress(data[:,2], data[:,3])
plt.plot(data[:,2], data[:,3], 'o', label='original data')
plt.plot(data[:,2], intercept + slope*data[:,2], 'r', label='fitted line')
plt.legend()
plt.xlabel('Power')
plt.ylabel('Price [$]')
plt.show()

print('correlation coefficient wrt the power:', r_value) #the power is the best feature!

In [ ]:
# linear regression with linregress (estimate price from km)

slope, intercept, r_value, p_value, std_err = stats.linregress(data[:,0], data[:,3])
plt.plot(data[:,0], data[:,3], 'o', label='original data')
plt.plot(data[:,0], intercept + slope*data[:,0], 'r', label='fitted line')
plt.legend()
plt.xlabel('Km')
plt.ylabel('Price [$]')
plt.show()

print('correlation coefficient wrt the km:', r_value) #km are not so good for the estimation

In [ ]:
# 2D linear regression with linear model (estimate price from year and power)

reg = sl.linear_model.LinearRegression().fit(data[:,1:3].reshape(-1,2), data[:,3].reshape(-1,1))
print ('slope coefficients:',reg.coef_)
print ('intercept:', reg.intercept_)

# 3D plot (blue real data, red estimated ones)

from mpl_toolkits.mplot3d import Axes3D
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
ax.scatter(data[:,1], data[:,2], data[:,3], zdir='z', s=5, c='b', depthshade=True)
ax.scatter(data[:,1], data[:,2], reg.intercept_+reg.coef_[:,0]*data[:,1]+reg.coef_[:,1]*data[:,2], zdir='z', s=5, c='r', depthshade=True)
plt.show()

print('correlation coefficient:', np.sqrt(reg.score(data[:,1:3].reshape(-1,2), data[:,3].reshape(-1,1))))
print('Much better correlation than the two features alone!')